<a href="https://colab.research.google.com/github/CathieLC/Wild_notebook_2021-2022/blob/main/Python_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests

# This part is common : we have to initiate a connection between Python and the SQL Database
import sqlite3

import pandas as pd

# Introduction

SQL is based on a client-server architecture. 


Python can connect with every SQL databases (Oracle MySQL, Microsoft, etc...). SQLite is a little bit particular : it is a little file, who can play the server role, without username and password. It will be helpful for this quest. But keep in mind that you could connect your Python script with other databases, following the same steps.

In [ ]:
# Here we just import the SQLite file. This part is specific for SQLite.

r = requests.get('https://raw.githubusercontent.com/murpi/wilddata/master/quests/toys_and_models.sqlite')
open('toys_and_models.sqlite', 'wb').write(r.content)

307200

In [ ]:
# This is the "connector". For secured database, it will be here that  you indicate your username and password.
conn = sqlite3.connect('toys_and_models.sqlite')
conn


In [ ]:
# This is the "cursor". A cursor is an object that can execute a query and retrieve the content of the result.
cursor = conn.cursor()
cursor

In [ ]:
# You can execute a query.
# The result is returned line by line. You can "fetch all" lines.
cursor.execute("select * from productlines").fetchall()

[('Classic Cars',
  'Attention car enthusiasts: Make your wildest car ownership dreams come true. Whether you are looking for classic muscle cars, dream sports cars or movie-inspired miniatures, you will find great choices in this category. These replicas feature superb attention to detail and craftsmanship and offer features such as working steering system, opening forward compartment, opening rear trunk with removable spare wheel, 4-wheel independent spring suspension, and so on. The models range in size from 1:10 to 1:24 scale and include numerous limited edition and several out-of-production vehicles. All models include a certificate of authenticity from their manufacturers and come fully assembled and ready for display in the home or office.',
  None,
  None),
 ('Motorcycles',
  'Our motorcycles are state of the art replicas of classic as well as contemporary motorcycle legends such as Harley Davidson, Ducati and Vespa. Models contain stunning details such as official logos, rotat

In [ ]:
# Pandas can import the result of a query
pd.DataFrame(cursor.execute("select * from productlines").fetchall())

,0,1,2,3
0,Classic Cars,Attention car enthusiasts: Make your wildest c...,None,None
1,Motorcycles,Our motorcycles are state of the art replicas ...,None,None
2,Planes,"Unique, diecast airplane and helicopter replic...",None,None
3,Ships,The perfect holiday or anniversary gift for ex...,None,None
4,Trains,Model trains are a rewarding hobby for enthusi...,None,None
5,Trucks and Buses,The Truck and Bus models are realistic replica...,None,None
6,Vintage Cars,Our Vintage Car models realistically portray a...,None,None


In [ ]:
# But it could be easier : you can use the pandas read_sql function.
pd.read_sql("select * from productlines", conn)

,productLine,textDescription,htmlDescription,image
0,Classic Cars,Attention car enthusiasts: Make your wildest c...,None,None
1,Motorcycles,Our motorcycles are state of the art replicas ...,None,None
2,Planes,"Unique, diecast airplane and helicopter replic...",None,None
3,Ships,The perfect holiday or anniversary gift for ex...,None,None
4,Trains,Model trains are a rewarding hobby for enthusi...,None,None
5,Trucks and Buses,The Truck and Bus models are realistic replica...,None,None
6,Vintage Cars,Our Vintage Car models realistically portray a...,None,None


In [ ]:
# Here is the schema of the database
df_table = pd.read_sql("SELECT * FROM sqlite_master where type = 'table'", conn)
df_table

,type,name,tbl_name,rootpage,sql
0,table,customers,customers,2,CREATE TABLE `customers` (\n `customerNumber`...
1,table,employees,employees,8,CREATE TABLE `employees` (\n `employeeNumber`...
2,table,offices,offices,10,CREATE TABLE `offices` (\n `officeCode` varch...
3,table,orderdetails,orderdetails,12,CREATE TABLE `orderdetails` (\n `orderNumber`...
4,table,orders,orders,49,CREATE TABLE `orders` (\n `orderNumber` int(1...
5,table,payments,payments,57,CREATE TABLE `payments` (\n `customerNumber` ...
6,table,productlines,productlines,64,CREATE TABLE `productlines` (\n `productLine`...
7,table,products,products,66,CREATE TABLE `products` (\n `productCode` var...


![Texte alternatif…](https://sp.mysqltutorial.org/wp-content/uploads/2009/12/MySQL-Sample-Database-Schema.png)

# It's your turn

- First you have to create a DataFrame with the 5 most ordered productcodes
- Then you have to create a second DataFrame with stocks by products
- Finally, you will make a merge (with Pandas) between both DataFrame to display the stocks of the 5 most ordered products 

In [ ]:
# les noms des tables sont afichés dans la colonne "name" du df_table ci-dessus

# j'affiche la table des commandes (je me servirais du productCode pour lier mes tables ensuite)
df_orders_best_Q = pd.read_sql("select * from orderdetails", conn)
df_orders_best_Q = df_orders_best_Q.groupby(by=["productCode"]).sum()
df_orders_best_Q = df_orders_best_Q.sort_values(by=['quantityOrdered'], ascending = False).iloc[0:5,:]
df_orders_best_Q

,orderNumber,quantityOrdered,priceEach,orderLineNumber
productCode,,,,
S18_3232,481242,1561,7171.59,334
S18_1342,256041,960,2284.94,161
S12_1108,256120,950,4644.32,179
S18_2949,256059,949,2334.21,146
S18_4600,256079,947,2702.50,151


In [ ]:
# j'affiche la table products pour avoir les "quantityInStock" et je me servirai du productCode pour merger mes tables
df_stocks_by_products = pd.read_sql("select * from products", conn)
df_stocks_by_products.head()


,productCode,productName,productLine,productScale,productVendor,productDescription,quantityInStock,buyPrice,MSRP
0,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.70
1,S10_1949,1952 Alpine Renault 1300,Classic Cars,1:10,Classic Metal Creations,Turnable front wheels; steering function; deta...,7305,98.58,214.30
2,S10_2016,1996 Moto Guzzi 1100i,Motorcycles,1:10,Highway 66 Mini Classics,"Official Moto Guzzi logos and insignias, saddl...",6625,68.99,118.94
3,S10_4698,2003 Harley-Davidson Eagle Drag Bike,Motorcycles,1:10,Red Start Diecast,"Model features, official Harley Davidson logos...",5582,91.02,193.66
4,S10_4757,1972 Alfa Romeo GTA,Classic Cars,1:10,Motor City Art Classics,Features include: Turnable front wheels; steer...,3252,85.68,136.00


In [ ]:
# Merge des deux df et affichage des 5 produits les plus vendus
df_final = pd.merge(df_orders_best_Q, df_stocks_by_products, on="productCode").sort_values(by=['quantityOrdered'], ascending = False).iloc[0:5,:]
df_final[['productCode','productName','quantityOrdered','priceEach','quantityInStock','buyPrice','productLine']]

,productCode,productName,quantityOrdered,priceEach,quantityInStock,buyPrice,productLine
0,S18_3232,1992 Ferrari 360 Spider red,1561,7171.59,8347,77.90,Classic Cars
1,S18_1342,1937 Lincoln Berline,960,2284.94,8693,60.62,Vintage Cars
2,S12_1108,2001 Ferrari Enzo,950,4644.32,3619,95.59,Classic Cars
3,S18_2949,1913 Ford Model T Speedster,949,2334.21,4189,60.78,Vintage Cars
4,S18_4600,1940s Ford truck,947,2702.50,3128,84.76,Trucks and Buses


Good, but not optimized. Your "merge" with pandas is pretty the same as the "join" in SQL. Moreover, your script has to download the whole product stocks table. 

So, now, you have to write a SQL query to have directly the stocks of the 5 most ordered products. (So, you have to "join" both tables in the SQL Query)

In [ ]:
# Your code here :
df_sql = pd.read_sql('''select products.productCode,productName,sum(quantityOrdered) as total_quantityOrdered ,priceEach,quantityInStock,buyPrice,productLine
              from products
              JOIN orderdetails
              ON products.productCode = orderdetails.productCode
              GROUP BY products.productCode
              ORDER BY total_quantityOrdered DESC
              LIMIT 5;''', conn)

df_sql

# FONCTIONNE AUSSI AVEC : "WHERE orderdetails.productCode = products.productCode;"

,productCode,productName,total_quantityOrdered,priceEach,quantityInStock,buyPrice,productLine
0,S18_3232,1992 Ferrari 360 Spider red,1561,143.94,8347,77.90,Classic Cars
1,S18_1342,1937 Lincoln Berline,960,88.36,8693,60.62,Vintage Cars
2,S12_1108,2001 Ferrari Enzo,950,166.24,3619,95.59,Classic Cars
3,S18_2949,1913 Ford Model T Speedster,949,100.30,4189,60.78,Vintage Cars
4,S18_4600,1940s Ford truck,947,115.03,3128,84.76,Trucks and Buses


Both methods have same results ? Congrats ! 

# Go forward

If you want, you can try to connect to the MySQL Server with your computer. To do this, you need to have MySQL Server on your computer, and to execute your python script from your computer too (so on Jupyterlab or on Spyder, **but not on google colab**).
Below, script to connect to your MySQL Server.
And here, [the SQL script to execute](https://drive.google.com/file/d/1wXu5t7bahLhbGXOT6uCTOVLjPH-2oQ_d/view?usp=sharing) in the workbench if you don't have the database.

In [ ]:
!pip install mysql-connector-python

In [ ]:
import mysql.connector
import pandas as pd

# Please replace by your user and your password
'''conn = mysql.connector.connect(user='YOUR_USERNAME', password='YOUR_PASSWORD',
                              host='localhost',
                              database='toys_and_models')

pd.read_sql('select * from orders', conn)'''

"conn = mysql.connector.connect(user='YOUR_USERNAME', password='YOUR_PASSWORD',\n                              host='localhost',\n                              database='toys_and_models')\n\npd.read_sql('select * from orders', conn)"

Or you can test with this online server:

In [ ]:
import mysql.connector
import pandas as pd

# Please replace by your user and your password
conn = mysql.connector.connect(user='toyscie', password='WILD4Rdata!',
                              host='51.68.18.102',port = '23456',
                              database='toys_and_models')

pd.read_sql('select * from orders', conn)

,orderNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber
0,10100,2019-09-21,2019-09-28,2019-09-25,Shipped,None,363
1,10101,2019-09-24,2019-10-03,2019-09-26,Shipped,Check on availability.,128
2,10102,2019-09-25,2019-10-03,2019-09-29,Shipped,None,181
3,10103,2019-10-14,2019-10-23,2019-10-18,Shipped,None,121
4,10104,2019-10-16,2019-10-25,2019-10-17,Shipped,None,141
...,...,...,...,...,...,...,...
278,10378,2021-10-26,2021-11-03,2021-10-27,Shipped,None,141
279,10379,2021-10-26,2021-11-03,2021-10-27,Shipped,None,141
280,10380,2021-11-01,2021-11-09,2021-11-03,Shipped,None,141
281,10381,2021-11-02,2021-11-10,2021-11-03,Shipped,None,321
